# Setup

In [1]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

### 1) Convert the code chunk found under section Divide an image into smaller patches using cropping into a function with the following signature:
crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color)

In [2]:
img = cv.imread('dog.jfif')
img_copy = img.copy()

def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    # img is the source image
    # num_horizontal_grid and num_vertical_grid are the number of patches along x and y axes.
    # line_color is the color of the grid line.
    height, width = img.shape[:2]
    num_horizontal_patches = num_horizontal_grid
    num_vertical_patches = num_vertical_grid
    
    # M and N are basically number of pixels per patch
    M, N = int(height/num_horizontal_patches), int(width/num_vertical_patches)

    x1, y1 = 0, 0
    
    for y in range(0, height, M):
        for x in range(0, width, N):
#         if (height - y) < M or (width - x) < N:
#             break
            
            y1 = y + M    # lower right coordinate that will be used to construct rectangle
            x1 = x + N
        
            # Check whether patch lower right coordinate exceeds image height and width
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
                
            # When patch lower right y-coordinate exceeds patch height
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            
            # When patch lower right x-coordinate exceeds patch width
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
        
            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)


crop_grid(img, 2, 3, (0, 255, 0))
cv.imshow('patched image', img_copy)
cv.waitKey(0)
cv.destroyAllWindows()

### 2) Display image sequences of smooth transition of two images with different values of  𝛼 . Refer to code in section "Image blending". Use "lena.jfif" and *"coins.jfif"* as the base images.

In [3]:
img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

dst = cv.addWeighted(img1, 0.35, img2, 0.65, 0)

cv.imshow('image1', img1)
cv.imshow('reshape_image2', img2)
cv.imshow('blend_image', dst)
cv.waitKey(0)
cv.destroyAllWindows()

### 3) Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.

In [4]:
!pip install imutils

In [5]:
import imutils as im
img = cv.imread('lena.jfif')

rotate = im.rotate_bound(img, -45)

cv.imshow('rotated_image', rotate)
cv.waitKey(0)
cv.destroyAllWindows()

### 4) Use the images with titles: "flower.jfif" and *"native-bee.png"*. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower.

In [6]:
img1= cv.imread('flower.jfif')
img2 = cv.imread('native-bee.png')

#resize image
img1 = cv.resize(img1, (500, 500), interpolation = cv.INTER_CUBIC)

#transformation
rows, cols = img1.shape[:2]
M = np.float32([[0.5, 0, 15], [0, 0.5, -35]])
dst = cv.warpAffine(img1, M, (cols, rows))

#conver to grayscale
img1_gray = cv.cvtColor(dst, cv.COLOR_BGR2GRAY)

#segment the flower
ret, mask = cv.threshold(img1_gray, 80, 255, cv.THRESH_BINARY_INV)
mask_inv = cv.bitwise_not(mask)

#apply mask
bee = cv.bitwise_and(img2, img2, mask = mask)
flower = cv.bitwise_and(dst, dst, mask = mask_inv)
result = cv.add(bee, flower)

#cv.imshow('flower', result)
cv.imshow('flower_bee', result)
cv.waitKey(0)
cv.destroyAllWindows()